<a href="https://colab.research.google.com/github/Tristan07999/TensorFlow2.0-with-tf.keras/blob/master/Movie_Reviews_(Text)_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#3RD SEP ZIQUAN LIN

#Using IMDB dataset contains the text of 50000 moive reviews 
#25000 reviews of training and 25000 reviews of testing
#For more advanced text classification, see MLCC text classification guide

from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np

try:
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

import tensorflow_hub as hub
import tensorflow_datasets as tfds

print('version:', tf.__version__)
print('eager mode:', tf.executing_eagerly())
print('hub version:', hub.__version__)
print('GPU is', 'available' if tf.test.is_gpu_available() else 'Not availavle')

!pip install -U tensorflow_hub==0.6.0

TensorFlow 2.x selected.
version: 2.0.0-rc0
eager mode: True
hub version: 0.6.0
GPU is Not availavle
Requirement already up-to-date: tensorflow_hub==0.6.0 in /usr/local/lib/python3.6/dist-packages (0.6.0)


In [0]:
#Download the IMDB dataset
train_validation_split = tfds.Split.TRAIN.subsplit([6, 4])

(train_data, validation_data), test_data = tfds.load(
  name = 'imdb_reviews', 
  split = (train_validation_split, tfds.Split.TEST), 
  as_supervised = True)

In [3]:
#Explore  and understand the format of data
#print first 10 examples
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch

<tf.Tensor: id=224, shape=(10,), dtype=string, numpy=
array([b"As a lifelong fan of Dickens, I have invariably been disappointed by adaptations of his novels.<br /><br />Although his works presented an extremely accurate re-telling of human life at every level in Victorian Britain, throughout them all was a pervasive thread of humour that could be both playful or sarcastic as the narrative dictated. In a way, he was a literary caricaturist and cartoonist. He could be serious and hilarious in the same sentence. He pricked pride, lampooned arrogance, celebrated modesty, and empathised with loneliness and poverty. It may be a clich\xc3\xa9, but he was a people's writer.<br /><br />And it is the comedy that is so often missing from his interpretations. At the time of writing, Oliver Twist is being dramatised in serial form on BBC television. All of the misery and cruelty is their, but non of the humour, irony, and savage lampoonery. The result is just a dark, dismal experience: the story p

In [4]:
#first 10 labels
train_labels_batch

<tf.Tensor: id=225, shape=(10,), dtype=int64, numpy=array([1, 1, 1, 1, 1, 1, 0, 1, 1, 0])>

In [5]:
#Build the model
#Three main architecture decisions: 
#How to represent the text?
#How many layers to use in the model?
#Hoe many hidden units to use for each layer?

#Input data: sentence; Labels 0 or 1
#One way to represent the text is to convert sentence into embeddeding vectors
#using pre-trained text emdedding model from TensorFlow Hub

embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

#This code was broken because of old version of tf_hub 0.5.0 so I installed 0.6.0 which is the new one

<tf.Tensor: id=405, shape=(3, 20), dtype=float32, numpy=
array([[ 3.9819887 , -4.4838037 ,  5.177359  , -2.3643482 , -3.2938678 ,
        -3.5364532 , -2.4786978 ,  2.5525482 ,  6.688532  , -2.3076782 ,
        -1.9807833 ,  1.1315885 , -3.0339816 , -0.7604128 , -5.743445  ,
         3.4242578 ,  4.790099  , -4.03061   , -5.992149  , -1.7297493 ],
       [ 3.4232912 , -4.230874  ,  4.1488533 , -0.29553518, -6.802391  ,
        -2.5163853 , -4.4002395 ,  1.905792  ,  4.7512794 , -0.40538004,
        -4.3401685 ,  1.0361497 ,  0.9744097 ,  0.71507156, -6.2657013 ,
         0.16533905,  4.560262  , -1.3106939 , -3.1121316 , -2.1338716 ],
       [ 3.8508697 , -5.003031  ,  4.8700504 , -0.04324996, -5.893603  ,
        -5.2983093 , -4.004676  ,  4.1236343 ,  6.267754  ,  0.11632943,
        -3.5934832 ,  0.8023905 ,  0.56146765,  0.9192484 , -7.3066816 ,
         2.8202746 ,  6.2000837 , -3.5709393 , -4.564525  , -2.305622  ]],
      dtype=float32)>

In [7]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation = 'relu')) #Rectified Linear Unit (ReLU)
model.add(tf.keras.layers.Dense(1, activation = 'sigmoid'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [0]:
#Complie the model
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])


In [11]:
#Train the model
history = model.fit(train_data.shuffle(10000).batch(512), epochs = 20, 
                    validation_data = validation_data.batch(512), verbose = 1)

Epoch 1/20
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


30/30 [==============================] - 7s 233ms/step - loss: 0.9057 - accuracy: 0.5295 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/20
30/30 [==============================] - 6s 209ms/step - loss: 0.6994 - accuracy: 0.6027 - val_loss: 0.6578 - val_accuracy: 0.6299
Epoch 3/20
30/30 [==============================] - 6s 207ms/step - loss: 0.6394 - accuracy: 0.6472 - val_loss: 0.6170 - val_accuracy: 0.6700
Epoch 4/20
30/30 [==============================] - 6s 211ms/step - loss: 0.5968 - accuracy: 0.6867 - val_loss: 0.5776 - val_accuracy: 0.7034
Epoch 5/20
30/30 [==============================] - 6s 210ms/step - loss: 0.5606 - accuracy: 0.7211 - val_loss: 0.5467 - val_accuracy: 0.7304
Epoch 6/20
30/30 [==============================] - 6s 211ms/step - loss: 0.5238 - accuracy: 0.7509 - val_loss: 0.5154 - val_accuracy: 0.7552
Epoch 7/20
30/30 [==============================] - 6s 211ms/step - loss: 0.4916 - accuracy: 0.7795 - val_loss: 0.4862 - val_accuracy: 0.7767
Epoch 8/2

In [13]:
#Evaluate the model
results = model.evaluate(test_data.batch(512), verbose = 0)
for name, value in zip(model.metrics_names, results):
  print('%s: %.3f' %(name, value))

loss: 0.310
accuracy: 0.868
